In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = 'Monthes/'

files = []
filesNoAdd = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(r, file))
            filesNoAdd.append(file)

for f in filesNoAdd:
    print(f)


April 2019.csv
February 2019.csv
January 2019.csv
March 2019.csv
May 2019.csv


In [3]:
numListOfBranch = ['401 Co','402 Co','404 Co','405 Co','412 Co','416 Co',
                   '417 Co','423 Co', '424 Co','425 Co','426 Co','429 Co','444 Co','490 Co']
listOfBranch = ['401 Co', '401.VMS cost', '401.V.S.P.', '401.AVG. WEEK',
                '402 Co', '402.VMS cost', '402.V.S.P.', '402.AVG. WEEK',
                '404 Co', '404.VMS cost', '404.V.S.P.', '404.AVG. WEEK',
                '405 Co', '405.VMS cost', '405.V.S.P.', '405.AVG. WEEK',
                '412 Co', '412.VMS cost', '412.V.S.P.', '412.AVG. WEEK',
                '416 Co', '416.VMS cost', '416.V.S.P.', '416.AVG. WEEK',
                '417 Co', '417.VMS cost', '417.V.S.P.', '417.AVG. WEEK',
                '423 Co', '423.VMS cost', '423.V.S.P.', '423.AVG. WEEK',
                '424 Co', '424.VMS cost', '424.V.S.P.', '424.AVG. WEEK',
                '425 Co', '425.VMS cost', '425.V.S.P.', '425.AVG. WEEK',
                '426 Co', '426.VMS cost', '426.V.S.P.', '426.AVG. WEEK',
                '429 Co', '429.VMS cost', '429.V.S.P.', '429.AVG. WEEK',
                '444 Co', '444.VMS cost', '444.V.S.P.', '444.AVG. WEEK',
                '490 Co', '490.VMS cost', '490.V.S.P.', '490.AVG. WEEK']

productDetail = ['Sku', 'UPC', 'Catalogue N', 'Title', 'Label' ,'Cost Price' ,'V.S.P.']
header = productDetail.copy()
header.extend(listOfBranch.copy())

In [4]:
def findID(sku, Data):
    numb = 0
    for SKUref in Data['Sku']:
        
        if str(sku) == str(SKUref):
            return numb
        numb += 1
    return "This good with Sku of " + sku + " is not in data." 

In [5]:
def buildList(num, ide, productDetail, listOfBranch, rawData):
    
    producList = []
    for det in productDetail:
        producList.append(rawData[det][ide])
    for branch in listOfBranch:
        branch = branch.replace(' Co','')
        head = str(branch + '.Sales Quantity')
        if head in rawData.head(0):
            try:
                producList.append(float(rawData[head][ide]))
                producList.append(float(rawData[head][ide]) * float(rawData['V.S.P.'][ide]))
                producList.append(float(rawData[head][ide]) * float(rawData['Cost Price'][ide]))
                producList.append(float(rawData[head][ide]) / float(num))
                
            except:
                print("A number in raw data is string")
                
    return producList

In [6]:
def listAddition(Dataide, ide, num, productDetail, listOfBranch, rawData, monthRawData):
    colid = 0
    for branch in listOfBranch:
        branch = branch.replace(' Co','')
        head = str(branch + '.Sales Quantity')
        if head in rawData.head(0):
            monthRawData.iloc[Dataide, colid+7] = monthRawData.iloc[Dataide, colid+7] + rawData[head][ide]
            monthRawData.iloc[Dataide, colid+8] = monthRawData.iloc[Dataide, colid+7] * float(rawData['V.S.P.'][ide])
            monthRawData.iloc[Dataide, colid+9] = monthRawData.iloc[Dataide, colid+7] * float(rawData['Cost Price'][ide])
            monthRawData.iloc[Dataide, colid+10]= monthRawData.iloc[Dataide, colid+7] / float(num)
        colid += 1
    return monthRawData

In [7]:
def noStock(sku,rawData):
    sku = str(sku)
    i = findID(sku, rawData)
    if np.sum(rawData.values[i,7:45]) > 0:
        return False
    else:
        return True


In [8]:
# a = np.zeros(shape=(1,len(header)))
monthRawData = pd.DataFrame(columns = list(header))
fcount = len(files)
b = []
for f in files:
#     f1 = (path + f)
    f1 = (f)
    rawData = pd.read_csv(f1 , encoding='latin1')
    sid =0
    b = 0
    for sku in rawData['Sku']:
        if sku not in list(monthRawData['Sku']):
            df = pd.DataFrame([list(buildList(fcount, sid, productDetail,
                                              listOfBranch, rawData))], columns= list(header))
            monthRawData = monthRawData.append(df)
        else:
            Dataide = findID(sku, monthRawData)
            b = listAddition(Dataide, sid, fcount, productDetail, listOfBranch, rawData, monthRawData)
        sid +=1

AssertionError: 63 columns passed, passed data had 7 columns

In [9]:
totalGoodSale = []
count = 0
for sku in monthRawData['Sku']:
    saleOfGood = []
    for numB in numListOfBranch:
        saleOfGood.append(monthRawData.iloc[count][numB])
    totalGoodSale.append(np.sum(saleOfGood))
    count += 1
monthRawData["total sale of Goods"] = totalGoodSale
status = []
for tot in totalGoodSale:
#     if tot == 0:
    if tot <= 0:
        status.append('Not moving')
    if (tot > 0) and (tot <=2):
        status.append('Slow moving')
    if tot > 2:
        status.append('Fast moving')
#
for j in range(len(filesNoAdd)):
    rawData = pd.read_csv(files[j])

    for i in range(len(status)):
#         print(i)
        try:
            if noStock(monthRawData.iloc[i]['Sku'],rawData):
                status[i] = 'Run out of stock'
        except:
            continue
#
monthRawData["Status"] = status



In [10]:
monthRawData.to_csv("AllMonths.csv", index=False)